In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
root_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(root_dir, 'data')

In [ ]:
merged_path = os.path.join(data_dir, 'gx_merged.csv')
merged_lags_path = os.path.join(data_dir, 'gx_merged_lags.csv')
volume_path = os.path.join(data_dir, 'gx_volume.csv')

In [ ]:
df_merged = pd.read_csv(merged_path)
df_merged_lags = pd.read_csv(merged_lags_path)
volume = pd.read_csv(volume_path)

In [ ]:
months = ['Jan', 'Feb', 'Mar','Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
test_merged = df_merged.loc[df_merged.test == 1, :]
test_merged_lags = df_merged_lags.loc[df_merged_lags.test == 1, :]
train_merged = df_merged.loc[df_merged.test == 0, :]
train_merged_lags = df_merged_lags.loc[df_merged_lags.test == 0, :]

In [ ]:
volume_before_1 = volume.loc[volume.month_num == -1, ['country', 'brand', 'month_num', 'month_name']]
volume_before_1.rename(columns={'month_num': 'month_num_1', 'month_name': 'month_name_1'}, inplace=True)
volume_before_1.head()

In [ ]:
def create_month(month_str, increment):
    idx = months.index(month_str)
    new_idx = (idx + increment)%12
    return months[new_idx]

In [ ]:
test_merged_volume = test_merged.merge(volume_before_1,  how='left', on=['country', 'brand'])
test_merged_volume.head()

In [ ]:
test_merged_volume['diff'] = test_merged_volume.month_num - test_merged_volume.month_num_1
test_merged_volume.head()

In [ ]:
test_merged_volume['month_name'] = np.vectorize(create_month)(test_merged_volume['month_name_1'], test_merged_volume['diff'])
del test_merged_volume['diff'], test_merged_volume['month_num_1'], test_merged_volume['month_name_1']
test_merged_volume.head()

In [ ]:
test_merged_lags_volume = test_merged_lags.merge(volume_before_1,  how='left', on=['country', 'brand'])
test_merged_lags_volume['diff'] = test_merged_lags_volume.month_num - test_merged_lags_volume.month_num_1
test_merged_lags_volume['month_name'] = np.vectorize(create_month)(test_merged_lags_volume['month_name_1'], test_merged_lags_volume['diff'])
del test_merged_lags_volume['diff'], test_merged_lags_volume['month_num_1'], test_merged_lags_volume['month_name_1']
test_merged_lags_volume.head(50)

In [ ]:
result_merged = pd.concat([train_merged, test_merged_volume])
result_merged_lags = pd.concat([train_merged_lags, test_merged_lags_volume])

In [ ]:
result_merged.to_csv(os.path.join(data_dir, 'gx_merged_months.csv'), index=False)
result_merged_lags.to_csv(os.path.join(data_dir, 'gx_merged_lags_months.csv'), index=False)

In [ ]:
print(len(result_merged))
print(len(df_merged))
print(len(result_merged_lags))
print(len(df_merged_lags))